# Proof of concept of ENUM-Based Regular ISD

This code applies ENUM-Based Regular ISD on random RSD instances, measures the success probability and list sizes and compares it with theoretical estimates.

The code has not been optimized, we recommend using it for moderate list sizes (e.g., $\leq 2^{15}$)

In [119]:
reset(); 

load("PGE.sage"); #required to perform Partial Gaussian Elimination
load("list_ops.sage"); #required to merge lists

#Generate random RSD instance (requires that n is divided by b)
def sample_rsdp_instance(n,k,b,w):

    F2 = GF(2);

    #Sample full rank parity-check matrix
    rank_H = 0;
    while rank_H < (n-k):
        H = random_matrix(F2,n-k,n);
        rank_H = rank(H);

    #Sample regular error vector
    e = matrix(F2,1,n);
    for i in range(w):
        pos = randrange(b);
        e[0,i*b+pos] = F2(1);

    #Compute syndrome
    s = H*e.transpose();

    return H, e, s;

##############################################

#Include the additional parity-check equations
def add_parity_checks(H, s, n, k, b, w):

    #Compute adapted parameters
    r = n - k;
    rp = n - k + w;
    kp = n - rp;

    ##Modify H: add partiy-check equations, remove last columns
    new_H = matrix(GF(2),rp,n);
    new_H[0:r, :] = H;
    for i in range(w):
        for j in range(b):
            new_H[r+i,i*b+j] = 1;

    new_H = new_H[:,0:n];

    ##Modify s: add new bits, corresponding to new parity-check equations
    new_s = matrix(GF(2), rp, 1);
    new_s[0:r, 0] = s;
    for i in range(w):
        new_s[r+i,0] = 1;

    return new_H, new_s;

# Theoretical estimates for ENUM-Based regular ISD
Returns the parameters to optimize the attack, as well internal quantities which will be later estimated with the simulations.

To deal with rounding issues, we are considering $\mathtt{v}\_\mathtt{f} = \left\lfloor \frac{k'+\ell}{w}\right\rfloor$ and $\mathtt{v}\_\mathtt{c} = \left\lceil \frac{k'+\ell}{w}\right\rceil$.
For the first list, we sample $\mathtt{v}\_\mathtt{f}$ coordinates from $\mathtt{w}\_\mathtt{f}\_\mathtt{left}$ blocks, and $\mathtt{v}\_\mathtt{c}$ coordinates from $\mathtt{w}\_\mathtt{c}\_\mathtt{left}$ blocks.
This way, the first list is built from vectors of length

$x_1 = \mathtt{v}\_\mathtt{f}\cdot \mathtt{w}\_\mathtt{c}\_\mathtt{left}+\mathtt{v}\_\mathtt{c}\cdot \mathtt{w}\_\mathtt{c}\_\mathtt{left}$

Analogous notation is used for the second list (the pedix $\mathtt{left}$ is replaced by $\mathtt{right}$ and the length of vectors is indicated by $x_2$). We choose these parameters so that $|x_1-x_2|$ is minized.

In [145]:
#Concrete cost estimate for enumeration based ISD
#it returns the parameters that optimize the attack, together with internal quantities:
#- parameters p and ell
#- success probability of one iteration
#- list size and number of collisions

#It takes also into account rounding issues

def enumeration_based_concrete_cost_rounding(n, k, w, b, S):

    
    k_prime = k-w;  #add parity-checks

    min_cost = 1000000000000000000000000000000;
    best_params = [];


    p_max = 30; 
    for p in range(0,min(p_max,w),2):

        for ell in range(1, 30):

            #Finding out optimal values for v_f, v_c, w_f and w_c
            v = (k_prime+ell)/w; #number of coordinates we sample from each block (ideally, without rounding issues)

            v_f = floor(v);
            v_c = ceil(v);

            w_f = w - (k_prime+ell-w*v_f);
            w_c = w - w_f;
            
            
            #Finding best choice for w_f_left, w_c_left, w_f_right and w_c_right
            w_c_left = -1;
            min_diff = n;
            for i_f_left in range(0,w_f+1):
                i_f_right = w_f-i_f_left;
                for i_c_left in range(0, w_c+1):
                    i_c_right = w_c - i_c_left;

                    num_left = i_f_left*v_f + i_c_left*v_c;
                    num_right = i_f_right*v_f + i_c_right*v_c;

                    if (abs(num_left - num_right)<min_diff)&(i_f_left+i_c_left == floor(w/2)):
                        min_diff = abs(num_left - num_right);
                        w_f_left = i_f_left;
                        w_f_right = i_f_right;
                        w_c_left = i_c_left;
                        w_c_right = i_c_right;
            
            if w_c_left<0: #check that the obtained optimal setting is well formed
                continue;
                
            #Lists sizes
            L1 = sum([binomial(w_f_left,i)*v_f^i*binomial(w_c_left, p-i)*v_c^(p-i) for i in range(p+1)]);
            L2 = sum([binomial(w_f_right,i)*v_f^i*binomial(w_c_right, p-i)*v_c^(p-i) for i in range(p+1)]);
            
            num_coll = L1*L2*2^(-ell);
            
            
            #success probability: we first compute probability for the first list
            p_iter_left_rounding = 0;
            for i in range(p+1):
                p_f = binomial(w_f_left,i)*(1-v_f/b)^(w_f_left-i)*(v_f/b)^i;
                p_c = binomial(w_c_left,p-i)*(1-v_c/b)^(w_c_left-(p-i))*(v_c/b)^(p-i);
                p_iter_left_rounding+=p_f*p_c;
            
            #success probability for second list
            p_iter_right_rounding = 0;
            for i in range(p+1):
                p_f = binomial(w_f_right,i)*(1-v_f/b)^(w_f_right-i)*(v_f/b)^i;
                p_c = binomial(w_c_right,p-i)*(1-v_c/b)^(w_c_right-(p-i))*(v_c/b)^(p-i);
                p_iter_right_rounding+=p_f*p_c;
            
            #overall success probability
            p_iter = min(1, p_iter_left_rounding*p_iter_right_rounding*S);
            
            #overall cost
            cost = N(log((n-k)^2*n+n*(L1+L2+num_coll),2)-log(p_iter,2));
            if cost>min_cost:
                continue;

            #update min cost  if new cost is smaller
            if cost<min_cost:
                min_cost = N(cost);
                best_params = [p, ell, v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right];
                internal_values = [N(log(L1,2)),N(log(L2,2)), N(log(num_coll,2)), N(p_iter)];
          
    return min_cost, best_params, internal_values

# Subroutine of ENUM-Based regular ISD

In [ ]:
#Enumerate regular vectors for initial lists

def all_regular_initial_list(p, w_f, v_f, w_c, v_c):
    
   # print(w_f, v_f, w_c, v_c);
    
    vec_length = w_f*v_f+w_c*v_c; #length of the vectors we are going to enumerate
    
    all_vecs = [];
    
    #we consider i_f blocks with length v_f and weight 1, i_c = p - i_f blocks with length v_c and weight 1
    for i_f in range(p+1):
        
        i_c = p - i_f;
        
        pos_v_f = cartesian_product([range(v_f) for i in range(i_f)]);
        pos_v_c = cartesian_product([range(v_c) for i in range(i_c)]);
        
        pos_unit_vectors_f = Combinations(w_f, i_f);
        pos_unit_vectors_c = Combinations(w_c, i_c);
        
        for pos_f in pos_unit_vectors_f:
            for pos_c in pos_unit_vectors_c:
                for pos_ones_f in pos_v_f:
                    for pos_ones_c in pos_v_c:
                        candidate = matrix(GF(2),1, vec_length);
                        
                        for j in range(len(pos_f)):
                            this_pos = pos_f[j]*v_f+pos_ones_f[j];
                            candidate[0, this_pos] = 1;
                        for j in range(len(pos_c)):
                            this_pos = w_f*v_f+v_c*pos_c[j]+pos_ones_c[j];
                            candidate[0, this_pos] = 1;
                            
                        all_vecs.append(candidate);
    
    return all_vecs;

######################################

#Create initial lists: takes as input the sub-parity check matrix and the list of vectors, 
#together with target syndrome and coeff_val (either 0 or 1)
def create_list_partial_sums(H_sub, list_vectors, target_s):

    list_sums = [];
    for x in list_vectors:
        s = target_s + x*H_sub.transpose();
        list_sums.append(s);

    return list_sums;
#######################################################

#Find collisions and return list of (merged) vectors
def merge_lists(list_sums_left, list_sums_right, all_vecs_left, all_vecs_right):
    
        
    indexes = colliding_indexes(list_sums_left, list_sums_right);    

    new_list_vectors = [];
    for i in range(len(indexes)):

        new_vector = block_matrix(GF(2),1,2,[all_vecs_left[indexes[i][0]], all_vecs_right[indexes[i][1]]]);
        new_list_vectors.append(new_vector);
    
    return new_list_vectors;


###################################################
#Sample regular permutation 
def ENUM_sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right):
    
   # print(v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right);
    P_list_left = sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, 0);
    P_list_right = sample_regular_permutation(n, b, v_f, v_c, w_f_right, w_c_right, b*(w_f_left + w_c_left));
    
    for i in P_list_right:
        P_list_left.append(i);
    
    
    #Now, place the remaining coordinates (i.e., those that will be move to the last n-k' positions)
    for i in range(n):
        if i not in P_list_left:
            P_list_left.append(i);
    
    #Create permutation matrix out of P
    P = matrix(GF(2),n,n);
    for i in range(n):
        P[P_list_left[i], i] = 1;
    

    return P;

##################################################

#Sample regular permutation
#Sample v_f coordinates from w_f blocks, v_c coordinates from w_c blocks and move them in first positions;
#These coordinates will constitute the information set
def sample_regular_permutation(n, b, v_f, v_c, w_f, w_c, offset):
    
    P_list = []; #regular permutation
    
    #We first sample coordinates for the w_f blocks (v_f from each block)
    for i in range(w_f):
        
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_f):
            P_list.append(i*b+perm_of_b[j]+offset);
    
    #Now, sample coordinates for the w_c blocks (v_c from each block)
    for i in range(w_c):
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_c):
            P_list.append((w_f+i)*b+perm_of_b[j]+offset);
    
    return P_list;

######################

#checks if input vector is regular
def check_regularity(x,b):
    n = x.ncols();
    
    is_regular = 1;
    for i in range(w):
        num_ones_in_block = x[0,i*b:(i+1)*b].list().count(1);
        if num_ones_in_block != 1:
            is_regular = 0;

    return is_regular;

# Implementation of ENUM-Based regular ISD
The implementation is only meant to be a proof of concept; it is only meant to verify the success probability, list sizes and the probabilities that the employed matrices have the desired rank 

In [156]:
######################
#ENUM-based ISD; it receives as input the H with additional equations
#It also updates the empirical estimates (last three parameters)
#It also requires the list of vectors to prepare the initial lists
#params must be formatted as [p, ell, w_f_left, w_c_left, w_f_right, w_c_right, v_f, v_c]
def ENUM_based_ISD(n, k_prime, w, b, new_H, new_s, all_vecs_left, all_vecs_right, params, num_success, num_full_rank, num_PGE_ok, avg_num_coll):
    
    p = params[0]; ell = params[1]; 
    w_f_left = params[2]; w_c_left = params[3]; w_f_right = params[4]; w_c_right = params[5]; 
    v_f = params[6]; v_c = params[7];
    
    ok = 0; #ok becomes 1 when a solution is found

    #Sample regular permutation
    P = ENUM_sample_regular_permutation(n, b, v_f, v_c, w_f_left, w_c_left, w_f_right, w_c_right);

    #Apply permutation to H
    perm_H = new_H*P;
    
    #Do PGE, continue only if PGE does not fail
    is_full_rank, reduced_H, reduced_s = PGE(n, n-k_prime, ell, perm_H, new_s);
    
    if is_full_rank:
        num_PGE_ok += 1;

        #create first list
        list_sums_left = create_list_partial_sums(reduced_H[0:ell,0:w_f_left*v_f+w_c_left*v_c], all_vecs_left, matrix(GF(2),1,ell));
        
        #create second list
        list_sums_right = create_list_partial_sums(reduced_H[0:ell,w_f_left*v_f+w_c_left*v_c:k-w+ell], all_vecs_right, reduced_s[0:ell].transpose());
        
        #find collisions and merge vectors
        merged_vecs = merge_lists(list_sums_left, list_sums_right, all_vecs_left, all_vecs_right);
        
        avg_num_coll += len(merged_vecs); #update estimate on average number of collisions
        
        ok_found = 0;
        for x in merged_vecs:
            H_sub = reduced_H[ell:,0:k_prime+ell];
            x_right = reduced_s[ell:] - H_sub*x.transpose();
            
            w_x = x_right.list().count(1);
            if w_x == w-2*p: #the vector has the right weight, continue by check
                
                perm_candidate_e = block_matrix(GF(2),1,2,[x, x_right.transpose()]);
                candidate_e = perm_candidate_e*P^-1;
                
                is_regular = check_regularity(candidate_e,b);
                if is_regular:
                    ok = 1; 
    
    num_success += ok;
    
    return num_success, num_full_rank, num_PGE_ok, avg_num_coll;

# Select the code parameters ($n$, $k$, $b$ and $w$) and the number of RSD instances to be generated (denoted by $\mathtt{num\_instances}$)

In [157]:
#Parameters
n = 160; #code length
k = 90; #code dimension
w = 10; #number of blocks with weight 1

num_instances = 100000; #number of RSD instances

b = n/w; #size of blocks (length of unit vectors forming the solution)

S = max(1,b**w/(2**(n-k))); #number of solutions

min_cost, params, quantities = enumeration_based_concrete_cost_rounding(n, k, w, b, S); #theoretical estimates

p = params[0]; ell = params[1]; v_f = params[2]; v_c = params[3]; w_f_left = params[4]; w_c_left = params[5]; w_f_right = params[6]; w_c_right = params[7];
L1 = quantities[0]; L2 = quantities[1]; num_coll = quantities[2]; p_iter = quantities[3];

print("Considering: [n, k, w, b] = "+str([n, k, w, b]));
print("Number of solutions: S = "+str(N(S)));
print("Optimal setting: p = "+str(p)+", ell = "+str(ell));
print("Th. success probability: p_iter = "+str(N(p_iter)));
print("Th. L1 = 2^"+str(L1)+", L2 = 2^"+str(L2)+", Th. Num Coll = 2^"+str(num_coll));
      

Considering: [n, k, w, b] = [160, 90, 10, 16]
Number of solutions: S = 1.00000000000000
Optimal setting: p = 2, ell = 10
Th. success probability: p_iter = 0.0702034493770043
Th. L1 = 2^9.66177809777199, L2 = 2^9.66177809777199, Th. Num Coll = 2^9.32355619554397


# Generating all vectors for enumeration
We generate all the vectors that we will later use for the enumeration; we do this once and then use these lists in every call to ENUM-based ISD

In [158]:
k_prime = k-w;

print("Composition of first list:");
print("-- w_f_left = "+str(w_f_left)+" blocks from which we pick v_f = "+str(v_f)+" coordinates");
print("-- w_c_left = "+str(w_c_left)+" blocks from which we pick v_c = "+str(v_c)+" coordinates");
print("-- total = "+str(w_f_left*v_f + w_c_left*v_c)+" coordinates");
print(" ");
print("Composition of second list:");
print("-- w_f_right = "+str(w_f_right)+" blocks from which we pick v_f = "+str(v_f)+" coordinates");
print("-- w_c_right = "+str(w_c_right)+" blocks from which we pick v_c = "+str(v_c)+" coordinates");
print("-- total = "+str(w_f_right*v_f + w_c_right*v_c)+" coordinates");
print(" ");
print("Total amount of coordinates = "+str(k_prime+ell));
print(" ");

all_vecs_left = all_regular_initial_list(p, w_f_left, v_f, w_c_left, v_c);
all_vecs_right = all_regular_initial_list(p, w_f_right, v_f, w_c_right, v_c);

#double check if enumeration has been done properly
num_vecs_left = sum([binomial(w_f_left,i)*v_f^i*binomial(w_c_left, p-i)*v_c^(p-i) for i in range(p+1)]);
num_vecs_right = sum([binomial(w_f_right,i)*v_f^i*binomial(w_c_right, p-i)*v_c^(p-i) for i in range(p+1)]);
print("First list: Expected size = "+str(num_vecs_left)+", Num of generated vectors = "+str(len(all_vecs_left)));
print("Second list: Expected size = "+str(num_vecs_right)+", Num of generated vectors = "+str(len(all_vecs_right)));

#Necessary later, when calling ENUM-based ISD
params = [p, ell, w_f_left, w_c_left, w_f_right, w_c_right, v_f, v_c]

Composition of first list:
-- w_f_left = 5 blocks from which we pick v_f = 9 coordinates
-- w_c_left = 0 blocks from which we pick v_c = 9 coordinates
-- total = 45 coordinates
 
Composition of second list:
-- w_f_right = 5 blocks from which we pick v_f = 9 coordinates
-- w_c_right = 0 blocks from which we pick v_c = 9 coordinates
-- total = 45 coordinates
 
Total amount of coordinates = 90
 
First list: Expected size = 810, Num of generated vectors = 810
Second list: Expected size = 810, Num of generated vectors = 810


# Start simulation and wait for results!

In [1]:
num_success = 0; #number of successful iterations
num_full_rank = 0; #num of full rank matrices in ISD
num_PGE_ok = 0; #number of H' with full ranks
avg_num_coll = 0; #average number of collisions

for id_instance in range(1, num_instances+1):

    #Sample RSD instance
    H, e, s = sample_rsdp_instance(n, k, b, w);

    #Add parity-chek equations
    new_H, new_s = add_parity_checks(H, s, n, k, b, w);
    
    if rank(new_H)==n-k_prime:
        num_full_rank +=1;
                
    #Launch ENUM-based ISD
    num_success, num_full_rank, num_PGE_ok, avg_num_coll = ENUM_based_ISD(n, k_prime, w, b, new_H, new_s, all_vecs_left, all_vecs_right, params, num_success, num_full_rank, num_PGE_ok, avg_num_coll);
        
    #Print current success probability
    if (id_instance%100) == 0:
        
        print("Num instances = "+str(id_instance));
        table_rows = [];
        table_rows.append(["P_iter",str(N(p_iter)),str(N(num_success/id_instance))]);
        table_rows.append(["Log2(Num_coll)",str(N(num_coll)),str(N(log(avg_num_coll/id_instance,2)))]);
        table_rows.append(["Cost",str(N(min_cost)),str(N(log((n-k)^2*n+n*(num_vecs_left+num_vecs_right+avg_num_coll/id_instance),2)-log(num_success/id_instance,2)))]);
        
        t = table(table_rows, header_row = [" ", " Theoretical","Empirical"])
        show(t)
        
        print("------------------------------");

NameError: name 'num_instances' is not defined